<a href="https://colab.research.google.com/github/takatakamanbou/Vision/blob/main/Vision2022_ex02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vision2022-ex02

課題の期限や提出の方法などについては，Visionチーム内に書いてます．

## 準備

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn
seaborn.set()

In [ ]:
#####  最小二乗法の正規方程式を解く関数
### solving the linear least squares problem (1)
#
# X.shape is assumed to be (N, D+1)
# y.shape is assumed to be (N,)
#
def solve(X, y):

    A = X.T @ X
    b = X.T @ y

    # x is the solution of the equation Ax = b
    x = np.linalg.solve(A, b)

    return x


### solving the linear least squares problem (2)
#  この関数については無視で構いません
def solve2(X, y):

    A = X.T @ X
    b = X.T @ y

    # rv[0] is the solution minimizing ||Ax - b||^2
    rv = np.linalg.lstsq(A, b)

    return rv[0]

In [ ]:
##### 1, x, x^2, x^3, ..., X^D をならべたデータ行列（N x (D+1)）をつくる
#     授業の説明のものとは転置の関係になっていることに注意
#
def makeDataMatrix(x, D):

    N = x.shape[0]
    X = np.zeros((N, D+1))
    X[:, 0] = 1
    for i in range(1, D+1):
        X[:, i] = x**i

    return X

## 課題A

以下のプログラムの中の関数 gendat は，正弦波にノイズをのせたデータを生成するものである．
この関数定義のセルとそれに続くセルを実行し，さらにこのプログラムの中身を解読して，次のことに答えなさい．
- 変数 `data_noisy` には何個のデータが格納されているか
- データのうち出力の正解の値には乱数を用いて生成したノイズが加えられている．
そのノイズはどんなものかを，関数名などを頼りに詳しく調べて答えなさい．

提出するもの： 実行結果のグラフと，上記について調べた結果

In [ ]:
def gendat(n, true_data=False, seed=0, sigma=0.0):

    if true_data:
        x = np.linspace(-0.1, 1.1, num=n)
        y = np.sin(2*np.pi*x)
    else:
        np.random.seed(seed)
        x = np.linspace(0.0, 1.0, num=n)
        y = np.sin(2*np.pi*x) + sigma*np.random.randn(n)

    return np.vstack((x, y)).T

In [ ]:
### 乱数の種(seed)を指定．適当に変えれば結果も変わる
seed = 0

data_noisy = gendat(30, seed=seed, sigma=0.1)
data_true  = gendat(1000, true_data=True)

fig, ax = plt.subplots(facecolor="white", figsize=(8, 6))
ax.set_xlim(-0.1, 1.1)
ax.set_ylim(-1.2, 1.2)
ax.plot(data_true[:, 0], data_true[:, 1], color="blue", label="true function")
ax.scatter(data_noisy[:, 0], data_noisy[:, 1], color="red", label="noisy data")
ax.legend()
plt.show()

## 課題B

以下のプログラムは，課題Aのプログラムを用いて生成したデータに D 次多項式をあてはめるものである．次のことをやりなさい．

1. 実行すると表示される mean squared error (MSE) が何を表しているか，プログラムを読解しなさい．
1. D を 1 から 20 まで変えたとき MSE の値やグラフがどのように変化するか観察しなさい．
1. 横軸を D，縦軸を学習データおよびテストデータの MSE としてグラフを描きなさい（Colab上でやってもよいし，他の手段を使ってもよい）．D が小さいときの MSE は他に比べてとても大きくなるので，縦軸の範囲を狭めてグラフを描き，D がいくつのときにテストデータの MSE が最小となっているか読み取れるようなものにすること．
その結果，Dが小さいときの点がグラフに描かれなくなってしまっても構わない．

提出するもの：上記の最後のステップで描いたグラフと，それに対する説明・考察．

**注意**
この課題と次の課題では，テストデータを使って次元数 $D$ や正則化パラメータ $\alpha$ の最適な値を決めようとしてます．授業で説明したように，未知のはずのテストデータを使うのは本当はいんちきです．



In [ ]:
##### generating the data

ndata = 20
sigma = 0.1

# learning (training) data
datL = gendat(ndata, seed=0, sigma=sigma)

# test data
datT = gendat(ndata, seed=1, sigma=sigma)

In [ ]:
##### fitting a D-th degree polynomial by least squares method

D = 1
XL = makeDataMatrix(datL[:, 0], D)
yL = datL[:, 1]
print('# X.shape =', XL.shape)
print('# y.shape =', yL.shape)
w = solve(XL, yL)
print('# estimated coefficients = ', w)

In [ ]:
##### mean squared error for the learning data
yL_est = XL @ w
msqeL = np.mean((yL - yL_est)**2)
print(f'# mean squared error (L) = {msqeL:.3e}')

##### mean squared error for the test data
XT = makeDataMatrix(datT[:, 0], D)
yT = datT[:, 1]
yT_est = XT @ w
msqeT = np.mean((yT - yT_est)**2)
print(f'# mean squared error (T) = {msqeT:.3e}')

In [ ]:
##### making the true (noiseless) data & more...
datTrue = gendat(1000, true_data=True)
X = makeDataMatrix(datTrue[:, 0], D)
y = datTrue[:, 1]
y_est = X @ w

##### plotting the results
fig, ax = plt.subplots(facecolor="white", figsize=(8, 6))
ax.set_xlim(-0.1, 1.1)
ax.set_ylim(-1.2, 1.2)
ax.plot(datTrue[:, 0], datTrue[:, 1], color="blue", label="true function")
ax.scatter(datL[:, 0], datL[:, 1], color="red", label="noisy data")
ax.plot(datTrue[:, 0], y_est, color="green", label="estimated curve")
ax.legend()
plt.show()

## 課題C

上記の問題を，$L_2$ 正則化を用いて解いてみよう．次のことをやりなさい．

1. 関数 solve をもとにして，正則化を考慮した関数 solve3 を作りなさい．solve3 は X, y に加えて alpha という名前の引数で正則化項の重み $\alpha$ の値を受け取るようにしよう．パラメータのうち定数の係数（バイアス項）には正則化を効かせないようにすること．
1. D を 20 として，$\alpha$ を様々に変えて実験を行い，横軸を $\alpha$，縦軸を学習データおよびテストデータに対する誤差としてグラフを描いてみよう．
    - $\alpha$ は $0$ 以上の任意の値をとるので，いくつか適当な値を選んで実行 → 結果を見て範囲を変える／狭める，というステップを何回か繰り返すのがよいかもしれない．とりあえず今回は [0, 0.001] からスタートしたらよい．
    - 余裕があれば，1つの $\alpha$ に対してノイズ生成の乱数の種を何通りか変えて，それらの誤差の平均でグラフを描くとなおよい
    - さらに余裕があれば，学習データの数を増減して同じ実験をやると傾向が変わるかどうか調べるとよい．
    - もっと余裕があれば，真の値に対する誤差も描いてみるとよい．

提出するもの：関数 solve3 の定義，描いたグラフとそれに対する説明・考察．